Load raw trial data

In [8]:
import pandas as pd 

data = pd.read_csv("data/raw_trials.csv")

In [9]:
print(data.columns)

Index(['title', 'objective', 'outcome_details', 'phase',
       'primary_completion_date', 'primary_endpoints_reported_date',
       'prior_concurrent_therapy', 'start_date', 'study_design',
       'treatment_plan', 'record_type', 'patients_per_site_per_month',
       'primary_endpoint_json', 'other_endpoint_json', 'associated_cro_json',
       'notes_json', 'outcomes_json', 'patient_dispositions_json',
       'results_json', 'study_keywords_json', 'tags_json',
       'primary_drugs_tested_json', 'other_drugs_tested_json',
       'therapeutic_areas_json', 'bmt_other_drugs_tested_json',
       'bmt_primary_drugs_tested_json', 'ct_gov_listed_locations_json',
       'ct_gov_mesh_terms_json'],
      dtype='object')


In [10]:
print(data.isna().sum().to_markdown())
print("Shape:", data.shape)

|                                 |   0 |
|:--------------------------------|----:|
| title                           |   0 |
| objective                       |   3 |
| outcome_details                 | 146 |
| phase                           |   0 |
| primary_completion_date         |  61 |
| primary_endpoints_reported_date | 161 |
| prior_concurrent_therapy        | 184 |
| start_date                      |  45 |
| study_design                    |  16 |
| treatment_plan                  |   1 |
| record_type                     |   0 |
| patients_per_site_per_month     | 119 |
| primary_endpoint_json           |   0 |
| other_endpoint_json             |   0 |
| associated_cro_json             |   0 |
| notes_json                      |   0 |
| outcomes_json                   |   0 |
| patient_dispositions_json       |   0 |
| results_json                    |   0 |
| study_keywords_json             |   0 |
| tags_json                       |   0 |
| primary_drugs_tested_json       

Generate unique hash per trial since trial id is missing
- i.e. "tid_0e8fa21079f928135dfc6164a15285f8"

In [11]:
import hashlib
import json
from pathlib import Path

OUTPUT_PATH = Path("cache/raw_trials_with_hash.csv")
OUTPUT_PATH.parent.mkdir(parents=True, exist_ok=True)

# ---------------------------------------------------------
# If file already exists → skip generation
# ---------------------------------------------------------
if OUTPUT_PATH.exists():
    print(f"⚠️ {OUTPUT_PATH} already exists — skipping hash generation.")
else:
    print("Generating raw_trials_with_hash.csv ...")

    def make_trial_hash(row):
        """
        Deterministic hash for a trial based on stable fields.
        You can add/remove fields if needed.
        """
        payload = {
            "title": row.get("title", ""),
            "start_date": row.get("start_date", ""),
            "phase": row.get("phase", ""),
        }
        raw = json.dumps(payload, sort_keys=True, ensure_ascii=False)
        return "tid_" + hashlib.md5(raw.encode("utf-8")).hexdigest()

    # Create trial_hash column
    data["trial_hash"] = data.apply(make_trial_hash, axis=1)

    # Move trial_hash to first column
    cols = ["trial_hash"] + [c for c in data.columns if c != "trial_hash"]
    data = data[cols]

    print(data.columns)
    print(data.shape)

    # Export
    data.to_csv(OUTPUT_PATH, index=False)
    print(f"✅ Saved to {OUTPUT_PATH}")

⚠️ cache/raw_trials_with_hash.csv already exists — skipping hash generation.


Using a chatbot, identify all interventions from each trial. For each intervention...
- label as the investigational product, active comparator, or placebo
- list all of the alternative names
- identify the molecular target 
- identify the mechanism of action

In [12]:
import re
import json
import time
import threading
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed

import pandas as pd
from services.openai_wrapper import OpenAIWrapper

# -------------------------------------------------
# CONFIG
# -------------------------------------------------
BASE_DIR = Path("cache")

TRIALS_WITH_HASH_CSV = Path("cache/raw_trials_with_hash.csv")

DRUG_ROLE_DIR = BASE_DIR / "trial_drug_roles"
DRUG_ROLE_DIR.mkdir(parents=True, exist_ok=True)

DRUG_ROLE_LOG_DIR = BASE_DIR / "trial_drug_roles_log"
DRUG_ROLE_LOG_DIR.mkdir(parents=True, exist_ok=True)

MASTER_ROLES_PATH = BASE_DIR / "trial_drug_roles_master.json"

MODEL = "gpt-5"
client = OpenAIWrapper()

MAX_WORKERS = 8

# Columns to feed into the chatbot
RELEVANT_COLS = [
    "title",
    "objective",
    "outcome_details",
    "notes_json",
    "results_json",
    "primary_drugs_tested_json",
    "other_drugs_tested_json",
    "therapeutic_areas_json",
    "bmt_other_drugs_tested_json",
    "bmt_primary_drugs_tested_json",
    "ct_gov_mesh_terms_json",
]

# -------------------------------------------------
# Helpers
# -------------------------------------------------
def extract_json_object(text: str) -> dict:
    """Extract first valid JSON object from model output."""
    if not isinstance(text, str):
        return {}
    text = text.strip()
    if not text:
        return {}

    # Direct parse first
    try:
        obj = json.loads(text)
        if isinstance(obj, dict):
            return obj
    except Exception:
        pass

    # Fallback: first {...} region
    m = re.search(r"\{.*\}", text, re.DOTALL)
    if not m:
        return {}
    try:
        obj = json.loads(m.group(0))
        if isinstance(obj, dict):
            return obj
    except Exception:
        return {}

    return {}


def build_prompt(trial_payload: dict) -> str:
    """
    Build prompt asking the LLM to:
    - Extract drug names
    - Canonicalize names by removing company/manufacturer/location qualifiers
    - Deduplicate synonymous names
    - For each canonical drug, return a dict with:
        * role (Investigational Product / Placebo / Active Comparator)
        * alternative_names (list)
        * molecular_target
        * mechanism
    """
    payload_json = json.dumps(trial_payload, ensure_ascii=False, indent=2)

    return f"""
You are a clinical trial design and interpretation expert.

You are given structured information about a clinical trial, including:
- Title and objective
- Study design and treatment plan
- JSON fields listing drugs tested in the study:
  - primary_drugs_tested_json
  - other_drugs_tested_json
  - bmt_other_drugs_tested_json
  - bmt_primary_drugs_tested_json

Your tasks:

1. Identify all DISTINCT physical drug entities explicitly used in the study.
   - Strings in the *_drugs_tested_json fields are drug-name candidates.
   - If these fields contain structured JSON, infer names from keys such as
     "name", "drug_name", "preferred_name", "label", etc.

2. Canonicalize each drug name:
   Remove company names, manufacturer qualifiers, geographic qualifiers,
   dosage-form qualifiers, or parenthetical descriptors that do NOT change
   the name of the underlying drug.
   Examples of correct canonicalization:
   - "Cinacalcet hydrochloride (Zhejiang Wansheng)" → "Cinacalcet hydrochloride"
   - "recombinant human thrombopoietin (rhTPO)" → "recombinant human thrombopoietin"
   - "Tofacitinib citrate (Zhejiang Wansheng Pharmaceutical Co)" → "Tofacitinib citrate"
   - "Xeljanz (tofacitinib, Pfizer)" → "Xeljanz"
   - "ABC-123 (CompanyX)" → "ABC-123"

   Keep only the essential drug or brand name as the canonical key.

3. Deduplicate synonymous names that refer to the SAME drug.
   - If multiple strings or variations clearly refer to the same physical drug,
     represent them with ONE canonical key.
   - Prefer the simplest, cleanest, shortest meaningful name for the key.
   - Collect other name variants as alternative names.

4. For EACH distinct (canonical, deduplicated) drug, build an object with
   the following FOUR fields:

   - "role": one of
       * "Investigational Product"
       * "Placebo"
       * "Active Comparator"

   - "alternative_names": a JSON array (list) of other names, codes, or labels
       that refer to the same drug in this trial.
       Examples:
       * recombinant erythropoietin stimulating protein → ["rESP", "NuPIAO"]
       * Xeljanz (tofacitinib, Pfizer) → ["tofacitinib"]

   - "molecular_target": using the trial title, objectives, and drug description,
       identify the main molecular target where possible
       (e.g., "TNF-α", "PD-1", "VEGF", "IL-6R", "CD20").
       If unknown or not inferable, use an empty string "".

   - "mechanism": a concise description of the pharmacologic or biologic mechanism,
       e.g., "TNF inhibitor", "monoclonal antibody", "kinase inhibitor",
       "bispecific antibody", "fusion protein", "G-CSF analog", etc.
       If unknown, use an empty string "".

Important rules:
- "role" MUST be exactly one of:
    "Investigational Product", "Placebo", "Active Comparator".
- Do NOT include any invented drugs not present in the structured fields.
- Combination therapies: label EACH component separately as an Investigational Product.
- Background or concomitant therapies should only be labeled if explicitly listed
  as drugs in the structured *_drugs_tested_json inputs.
- alternative_names may be empty ([]) if there are no clear synonyms.

Input JSON:
{payload_json}

Output format (CRITICAL):
- Return ONLY a valid JSON object (no extra commentary) with:
    - keys   = canonical, deduplicated drug names (no company/manufacturer tags)
    - values = objects with EXACTLY the following four fields:
        * "role"               (string)
        * "alternative_names"  (array of strings)
        * "molecular_target"   (string)
        * "mechanism"          (string)

- Example output:
{{
  "Cinacalcet hydrochloride": {{
    "role": "Investigational Product",
    "alternative_names": ["Cinacalcet", "Sensipar"],
    "molecular_target": "Calcium-sensing receptor",
    "mechanism": "Calcimimetic; positive allosteric modulator of the calcium-sensing receptor"
  }},
  "Xeljanz": {{
    "role": "Active Comparator",
    "alternative_names": ["tofacitinib"],
    "molecular_target": "JAK1 and JAK3",
    "mechanism": "Janus kinase (JAK) inhibitor"
  }},
  "Placebo": {{
    "role": "Placebo",
    "alternative_names": [],
    "molecular_target": "",
    "mechanism": "Inert comparator with no active pharmacologic effect"
  }}
}}

Before you return the JSON:
- Double-check that:
  * Every value is an object with ALL FOUR fields.
  * "role" uses ONLY the allowed strings.
  * alternative_names is ALWAYS a JSON array (list).
  * No drug key contains company/manufacturer/location qualifiers in parentheses.
  * You have not invented any drugs not present in the structured inputs.
""".strip()


# Shared counters & master mapping
counter = {
    "processed": 0,
    "skipped_existing": 0,
    "llm_error": 0,
    "parse_error": 0,
}
counter_lock = threading.Lock()

master_roles: dict[str, dict] = {}
master_lock = threading.Lock()


def process_trial(row: dict, idx: int, total: int) -> None:
    """Process one trial: prompt LLM, save output & log (only if valid)."""
    trial_hash = str(row.get("trial_hash", "")).strip()
    if not trial_hash:
        print(f"⚠️ [{idx}/{total}] Missing trial_hash, skipping")
        return

    out_fp = DRUG_ROLE_DIR / f"{trial_hash}.json"
    if out_fp.exists():
        with counter_lock:
            counter["skipped_existing"] += 1
        return

    # Build payload from selected columns
    trial_payload = {"trial_hash": trial_hash}
    for col in RELEVANT_COLS:
        trial_payload[col] = row.get(col, "")

    prompt = build_prompt(trial_payload)

    token = trial_hash
    hash_id = trial_hash

    text_response = ""
    raw_response = None
    total_cost = 0.0
    elapsed = 0.0

    # Call LLM
    try:
        t0 = time.perf_counter()
        res = client.query(prompt=prompt, model=MODEL)
        elapsed = round(time.perf_counter() - t0, 2)

        text_response = (res.get("text_response") or "").strip()
        raw_response = res.get("raw_response")
        total_cost = float(res.get("cost") or 0.0)

    except Exception as e:
        print(f"⚠️ [{idx}/{total}] LLM error for trial_hash={trial_hash}: {e}")
        with counter_lock:
            counter["llm_error"] += 1
        return

    drug_roles = extract_json_object(text_response)

    # Treat non-dict OR empty dict as invalid → do NOT save anything
    if not isinstance(drug_roles, dict) or not drug_roles:
        print(f"⚠️ [{idx}/{total}] JSON parse/validity error trial_hash={trial_hash}, raw={text_response!r}")
        with counter_lock:
            counter["parse_error"] += 1
        return

    mapped = {
        "trial_hash": trial_hash,
        "title": row.get("title"),
        "drug_roles": drug_roles,
        "source": "llm",
    }

    # Save per-trial roles JSON
    out_fp.write_text(json.dumps(mapped, ensure_ascii=False, indent=2), encoding="utf-8")

    # Log entry
    log_payload = {
        "token": token,
        "hash_id": hash_id,
        "model": MODEL,
        "prompt": prompt,
        "structured_response": json.dumps(mapped, ensure_ascii=False, indent=2),
        "raw_response": repr(raw_response),
        "total_cost": total_cost,
        "time_elapsed": elapsed,
    }
    (DRUG_ROLE_LOG_DIR / f"{hash_id}.json").write_text(
        json.dumps(log_payload, ensure_ascii=False, indent=2),
        encoding="utf-8",
    )

    # Update master roles
    with master_lock:
        master_roles[trial_hash] = mapped
        MASTER_ROLES_PATH.write_text(
            json.dumps(master_roles, ensure_ascii=False, indent=2),
            encoding="utf-8"
        )

    with counter_lock:
        counter["processed"] += 1
        if counter["processed"] % 50 == 0:
            print(f"Progress: processed {counter['processed']} trials...")


# -------------------------------------------------
# RUN CONCURRENTLY
# -------------------------------------------------
df_trials = pd.read_csv(TRIALS_WITH_HASH_CSV, dtype=str).fillna("")
rows = df_trials.to_dict(orient="records")
total_trials = len(rows)
print(f"Loaded {total_trials} trials from {TRIALS_WITH_HASH_CSV}")

with ThreadPoolExecutor(max_workers=MAX_WORKERS) as ex:
    futures = {
        ex.submit(process_trial, row, idx, total_trials): row.get("trial_hash")
        for idx, row in enumerate(rows, start=1)
    }
    for fut in as_completed(futures):
        th = futures[fut]
        try:
            fut.result()
        except Exception as e:
                print(f"⚠️ Worker error trial_hash={th}: {e}")

print(
    f"✅ Trial drug-role mapping complete. "
    f"processed={counter['processed']}, "
    f"skipped={counter['skipped_existing']}, "
    f"llm_error={counter['llm_error']}, "
    f"parse_error={counter['parse_error']}"
)
print(f"Roles directory: {DRUG_ROLE_DIR}")
print(f"Log directory:   {DRUG_ROLE_LOG_DIR}")
print(f"Master roles:    {MASTER_ROLES_PATH}")

Loaded 184 trials from cache/raw_trials_with_hash.csv
✅ Trial drug-role mapping complete. processed=2, skipped=182, llm_error=0, parse_error=0
Roles directory: cache/trial_drug_roles
Log directory:   cache/trial_drug_roles_log
Master roles:    cache/trial_drug_roles_master.json


In [7]:
import json
from pathlib import Path

LOG_DIR = Path("cache/trial_drug_roles_log")

total_cost = 0.0
num_entries = 0
costs = []

for fp in LOG_DIR.glob("*.json"):
    try:
        log = json.loads(fp.read_text(encoding="utf-8"))
        c = float(log.get("total_cost") or 0.0)
        total_cost += c
        costs.append((fp.name, c))
        num_entries += 1
    except Exception as e:
        print(f"⚠️ Error reading {fp.name}: {e}")

# Sort descending by cost
costs_sorted = sorted(costs, key=lambda x: x[1], reverse=True)

print("========== LLM COST SUMMARY ==========")
print(f"Total LLM cost:             ${total_cost:,.4f}")
print(f"Number of logged trials:     {num_entries}")
if num_entries > 0:
    print(f"Average cost per trial:      ${total_cost / num_entries:,.4f}")
print("")

print("Top 10 most expensive trials:")
for name, c in costs_sorted[:10]:
    print(f"  {name}: ${c:,.4f}")

print("========================================")

========== LLM COST SUMMARY ==========
Total LLM cost:             $4.0495
Number of logged trials:     182
Average cost per trial:      $0.0222

Top 10 most expensive trials:
  tid_94883aa2d583afced004e22a7991ef3e.json: $0.0474
  tid_e0a77c4ecf93cf781f04cc467c974511.json: $0.0442
  tid_1158b3369546dc4b16dc21c8c026b619.json: $0.0431
  tid_261f0233308ca080d1c60e3fda61ca85.json: $0.0431
  tid_9727cefa81bf0a9c341273bce42d3346.json: $0.0410
  tid_8b4d60a5fddc078962af34399d7e342c.json: $0.0409
  tid_5646e008947e9cfd7d19f09c2e982fa4.json: $0.0406
  tid_763e3011bc90e46c88c7a2953a39ed2a.json: $0.0406
  tid_fdfe189c015d6ba853db7988409431ef.json: $0.0387
  tid_7e80effdd579ba535ef686ac50dcc4bc.json: $0.0382


In [13]:
import json

import pandas as pd

# -------------------------------------------------
# Build trial_product_breakdown.csv
# -------------------------------------------------
OUT_CSV = BASE_DIR / "trial_product_breakdown.csv"

rows = []

for fp in DRUG_ROLE_DIR.glob("*.json"):
    try:
        obj = json.loads(fp.read_text(encoding="utf-8"))
    except Exception as e:
        print(f"⚠️ Error reading {fp.name}: {e}")
        continue

    trial_hash = obj.get("trial_hash")
    if not trial_hash:
        print(f"⚠️ Missing trial_hash in {fp.name}, skipping")
        continue

    drug_roles = obj.get("drug_roles") or {}
    if not isinstance(drug_roles, dict):
        print(f"⚠️ drug_roles not dict in {fp.name}, skipping")
        continue

    # Containers
    inv_names = []
    inv_alt_names = []          # list of lists
    inv_targets = []
    inv_mechanisms = []

    ac_names = []
    ac_alt_names = []           # list of lists
    ac_targets = []
    ac_mechanisms = []

    plc_names = []
    plc_alt_names = []          # list of lists
    plc_targets = []
    plc_mechanisms = []

    for drug_name, meta in drug_roles.items():
        if not isinstance(meta, dict):
            continue

        role = (meta.get("role") or "").strip()
        role_norm = role.lower()

        alt_names = meta.get("alternative_names") or []
        if not isinstance(alt_names, list):
            alt_names = [str(alt_names)]

        molecular_target = meta.get("molecular_target") or ""
        mechanism = meta.get("mechanism") or ""

        if role_norm == "investigational product":
            inv_names.append(drug_name)
            inv_alt_names.append(alt_names)
            inv_targets.append(molecular_target)
            inv_mechanisms.append(mechanism)
        elif role_norm == "active comparator":
            ac_names.append(drug_name)
            ac_alt_names.append(alt_names)
            ac_targets.append(molecular_target)
            ac_mechanisms.append(mechanism)
        elif role_norm == "placebo":
            plc_names.append(drug_name)
            plc_alt_names.append(alt_names)
            plc_targets.append(molecular_target)
            plc_mechanisms.append(mechanism)

    rows.append(
        {
            "trial_hash": trial_hash,

            "investigational_products": inv_names,
            "investigational_products_alternative_names": inv_alt_names,
            "investigational_products_molecular_target": inv_targets,
            "investigational_products_mechanism": inv_mechanisms,

            "active_comparators": ac_names,
            "active_comparators_alternative_names": ac_alt_names,
            "active_comparators_molecular_target": ac_targets,
            "active_comparators_mechanism": ac_mechanisms,

            "placebos": plc_names,
            "placebos_alternative_names": plc_alt_names,
            "placebos_molecular_target": plc_targets,
            "placebos_mechanism": plc_mechanisms,
        }
    )

df_out = pd.DataFrame(rows).sort_values("trial_hash")

OUT_CSV.parent.mkdir(parents=True, exist_ok=True)
df_out.to_csv(OUT_CSV, index=False)

print(f"Saved trial product breakdown to {OUT_CSV}")
print(df_out.head().to_markdown())

Saved trial product breakdown to cache/trial_product_breakdown.csv
|     | trial_hash                           | investigational_products     | investigational_products_alternative_names                                                                                              | investigational_products_molecular_target   | investigational_products_mechanism                                                                      | active_comparators   | active_comparators_alternative_names                                                | active_comparators_molecular_target     | active_comparators_mechanism                                                                        | placebos   | placebos_alternative_names   | placebos_molecular_target   | placebos_mechanism   |
|----:|:-------------------------------------|:-----------------------------|:----------------------------------------------------------------------------------------------------------------------------------------|

#### Identify the molecular target for each investigational product